Importing libraries and setting global variables

In [3]:
import pandas as pd
import json
import os
import google.generativeai as genai
from bs4 import BeautifulSoup
import subprocess
import Levenshtein as lev

/Users/jedtoner1/Desktop/Developer/PROPERTY/property_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Suppress the SettingWithCopyWarning
pd.options.mode.chained_assignment = None

PROPERTY_CSV = 'domain_properties.csv' #obtained from https://www.kaggle.com/datasets/alexlau203/sydney-house-prices
RANKINGS_URL = 'https://www.domain.com.au/liveable-sydney/sydneys-most-liveable-suburbs-2019/sydneys-569-suburbs-ranked-for-liveability-2019-903130/'
API_KEY = os.getenv('API_KEY')

Loading the property dataset, obtained from Kaggle with link above. We use Gemini API to add postcodes to all of the suburbs - since this process  can take some time we save the results for ease of continuous use

In [5]:
class PropertyDataLoader:

    def __init__(self, data_path):
        self.data_path = data_path
        self.data = self.load_data(data_path)
        self.suburbs = self.data['suburb'].unique().tolist()
        self.add_postcodes(load = True)

    def load_data(self, data_path):

        raw_df = pd.read_csv(data_path)
        raw_df.loc[:, 'date_sold'] = pd.to_datetime(raw_df['date_sold'], format="mixed", dayfirst=True)
        raw_df = raw_df[raw_df['type'] != 'Vacant land'] # remove any land vacanicies will not be used in model

        return raw_df
    
    def add_postcodes(self, save = False, load = False):
        if load:
            suburb_postcodes = json.loads(open('property_suburb_postcode_mapping.json').read())
        else:
            suburbs = self.suburbs
            prompt = f"""
            I have a list of suburbs: {str(suburbs)}
            Get the postcode of each suburb.
            Format your response in the form [suburb: postcode] as a json string.
            Ensure the response uses the exact suburb names as provided in the input.
            Ensure no additional formatting is present in the response. It should start and end with curly braces.
            """
            genai.configure(api_key=API_KEY)
            model = genai.GenerativeModel('gemini-1.5-pro')
            suburb_postcodes = model.generate_content(prompt).text
            if save:
                with open('property_suburb_postcode_mapping.json', 'w') as f:
                    f.write(suburb_postcodes)
            suburb_postcodes = json.loads(suburb_postcodes)
        
        self.data['postcode'] = self.data['suburb'].map(suburb_postcodes)

To get a better understanding of suburb quality and importance of location in property price prediction - we scrape the Domain suburb rankings for sydney suburbs. This is obtained from the URL above where suburbs are ranked on 19 different indicators ranging from traffic, access to schools, cafes and proximity to CBD

In [6]:
class SuburbRankingLoader:
    
    def __init__(self, url):
        self.url = url
        self.data = self.load_data(url)
        self.suburbs = self.data['Suburb'].tolist()
        self.add_postcodes(load = True)

    def load_data(self, url):
        
        command = [
            'curl',
            '-A', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
            url
        ]

        result = subprocess.run(command, capture_output=True, text=True)

        soup = BeautifulSoup(result.stdout, 'html.parser')

        rankings = []

        suburb_headings = soup.find_all('h3')
        for heading in suburb_headings:
            text = heading.text.strip()
            if text.startswith(tuple(str(i) for i in range(1, 570))):  # Check if heading starts with a number
                rank, suburb = text.split('.', 1)
                rankings.append((int(rank), suburb.strip()))
        
        rankings.sort(key=lambda x: x[0])

        ranking_df = pd.DataFrame(rankings, columns=['Rank', 'Suburb'])

        return ranking_df
    
    def add_postcodes(self, save = False, load = False):
        if load:
            suburb_postcodes = json.loads(open('ranking_suburb_postcode_mapping.json').read())
        else:
            suburbs = self.suburbs
            prompt = f"""
            I have a list of suburbs: {str(suburbs)}
            Get the postcode of each suburb.
            Format your response in the form [suburb: postcode] as a json string.
            Ensure the response uses the exact suburb names as provided in the input.
            Ensure no additional formatting is present in the response. It should start and end with curly braces.
            """
            genai.configure(api_key=API_KEY)
            model = genai.GenerativeModel('gemini-1.5-pro')
            suburb_postcodes = model.generate_content(prompt).text
            print(suburb_postcodes)
            if save:
                with open('ranking_suburb_postcode_mapping.json', 'w') as f:
                    f.write(suburb_postcodes)
            suburb_postcodes = json.loads(suburb_postcodes)
        
        self.data['postcode'] = self.data['Suburb'].map(suburb_postcodes)
        self.data['postcode'] = self.data['postcode'].astype(int)

We then combine this data into one csv for futher analysis in R

In [7]:
def combine_data(property_df, review_df):

    # Merge the property and review data on postcode
    merged_df = pd.merge(property_df, review_df, on = 'postcode', how='left')
    merged_df['str_distance'] = merged_df.apply(lambda row: lev.distance(row['suburb'].lower(), row['Suburb'].lower()) if pd.notnull(row['Suburb']) else None, axis=1)
    merged_df['min_str_distance'] = merged_df.groupby('postcode')['str_distance'].transform('min')
    merged_df = merged_df[merged_df['str_distance'] == merged_df['min_str_distance']]
    merged_df = merged_df.drop(columns=['str_distance', 'min_str_distance', 'Suburb', 'postcode'])
    merged_df = merged_df.rename(columns={'Rank': 'suburb_ranking'})

    return merged_df

Running the code

In [10]:
property_loader = PropertyDataLoader(PROPERTY_CSV)
property_df = property_loader.data

ranking_loader = SuburbRankingLoader(RANKINGS_URL)
ranking_df = ranking_loader.data

master_df = combine_data(property_df, ranking_df)

master_df.to_csv('master_dataset.csv', index=False)

master_df

,price,date_sold,suburb,num_bath,num_bed,num_parking,property_size,type,suburb_population,suburb_median_income,suburb_sqkm,suburb_lat,suburb_lng,suburb_elevation,cash_rate,property_inflation_index,km_from_cbd,suburb_ranking
4,452000,2016-01-13 00:00:00,Leumeah,1,3,1,344,House,9835,32292,4.055,-34.05375,150.83957,81,2.0,150.9,40.12,334.0
17,495000,2016-01-15 00:00:00,Leumeah,1,3,2,582,House,9835,32292,4.055,-34.05375,150.83957,81,2.0,150.9,40.12,334.0
30,890000,2016-01-18 00:00:00,Picnic Point,2,4,3,715,House,6160,40560,3.859,-33.97301,151.00632,33,2.0,150.9,22.31,403.0
34,533000,2016-01-18 00:00:00,Whalan,3,4,2,695,House,5973,24180,2.429,-33.75572,150.80361,37,2.0,150.9,39.53,453.0
42,1120500,2016-01-19 00:00:00,North Rocks,2,4,2,904,House,7965,40092,5.462,-33.77572,151.01474,92,2.0,150.9,20.61,271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35893,1900000,2021-12-30 00:00:00,Mascot,3,4,2,285,House,14772,41912,11.961,-33.94660,151.18371,3,0.1,220.1,9.35,520.0
35896,1900000,2021-12-31 00:00:00,Kellyville,3,4,2,540,House,27971,46228,18.645,-33.69583,150.95622,78,0.1,220.1,30.08,481.0
35898,1300000,2021-12-31 00:00:00,Seven Hills,3,7,2,1208,House,19326,33540,9.629,-33.77743,150.94272,38,0.1,220.1,26.58,370.0
35901,1025000,2021-12-31 00:00:00,Sydney,2,2,1,129,Apartment / Unit / Flat,17252,35412,2.940,-33.86794,151.20998,65,0.1,220.1,0.31,3.0
